# VacationPy

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [2]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
path=r"C:\Users\Intel NUC\OneDrive\++Tec Data Analytics Bootcamp\HWKS"
api_keys=os.path.join(path,r'api_keys.py')

def read_key(api_keys):
    api_keys_file = open(api_keys,"r")
    g_key= api_keys_file.readline().rstrip()
    return g_key

g_key=read_key(api_keys)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
file='../output_data/cities.csv'
cities_df=pd.read_csv(file)
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mataura,93,NZ,1603598556,92,-46.19,168.86,12.22,2.24
1,Lagoa,94,PT,1603598556,71,39.05,-27.98,17.92,7.47
2,Sinazongwe,22,ZM,1603598557,39,-17.26,27.46,24.40,0.91
3,Jamestown,90,US,1603598428,86,42.10,-79.24,4.44,2.35
4,Ushuaia,75,AR,1603598442,65,-54.80,-68.30,7.00,4.10
...,...,...,...,...,...,...,...,...,...
565,Kushiro,75,JP,1603598793,44,42.98,144.37,14.00,4.10
566,Mocuba,0,MZ,1603598794,83,-16.84,36.99,21.25,0.57
567,Cantaura,66,VE,1603598795,90,9.31,-64.36,23.06,3.28
568,Pacific Grove,90,US,1603598795,82,36.62,-121.92,16.00,2.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

In [10]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=cities_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
ideal_weather_cities=cities_df.loc[cities_df["Max Temp"]>25,:]
ideal_weather_cities=ideal_weather_cities.loc[cities_df["Max Temp"]<30,:]
ideal_weather_cities=ideal_weather_cities.loc[cities_df["Wind Speed"]<3,:]
hotel_df=ideal_weather_cities.loc[cities_df["Cloudiness"]==0,:]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
15,Ambilobe,0,MG,1603598425,56,-13.20,49.05,25.39,2.29
28,Yanam,0,IN,1603598565,62,16.73,82.22,29.00,2.60
246,Cabo San Lucas,0,MX,1603598354,77,22.89,-109.91,27.78,0.45
453,Fuerte Olimpo,0,PY,1603598739,72,-21.04,-57.87,25.61,1.72
490,Ramnagar,0,IN,1603598757,37,29.40,79.12,25.92,0.22
528,Boa Vista,0,BR,1603598740,66,2.82,-60.67,29.00,2.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df["Hotel Name"]=""
hotel_df

<ipython-input-14-4b7c85aad61d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
15,Ambilobe,0,MG,1603598425,56,-13.20,49.05,25.39,2.29,
28,Yanam,0,IN,1603598565,62,16.73,82.22,29.00,2.60,
246,Cabo San Lucas,0,MX,1603598354,77,22.89,-109.91,27.78,0.45,
453,Fuerte Olimpo,0,PY,1603598739,72,-21.04,-57.87,25.61,1.72,
490,Ramnagar,0,IN,1603598757,37,29.40,79.12,25.92,0.22,
528,Boa Vista,0,BR,1603598740,66,2.82,-60.67,29.00,2.10,


In [17]:
coordinates=[]

lat_list=hotel_df["Lat"].tolist()
lng_list=hotel_df["Lng"].tolist()
cities_list=hotel_df["City"].tolist()

for x in range(len(hotel_df["Lat"])):
    coordinates.append(f'{lat_list[x]},{lng_list[x]}')

coordinates

['-13.2,49.05',
 '16.73,82.22',
 '22.89,-109.91',
 '-21.04,-57.87',
 '29.4,79.12',
 '2.82,-60.67']

In [18]:
# Use Google Places API to search for hotels in the cities.

results=[]

def search_hotels(google_location):
    params = {
        "location": google_location,
        "type" : "lodging",
        "radius": 5000,
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
   
    return response.json()

for x in range(len(hotel_df["City"])):
    try:
        google_location=coordinates[x]
        print(f'Processing research for city: {cities_list[x]}')
        search_result=search_hotels(google_location)
        results.append(search_result)
    except:
        print(f'Research for hotels in the city: {cities_list[x]} was not succesful')

Processing research for city: Ambilobe
Processing research for city: Yanam
Processing research for city: Cabo San Lucas
Processing research for city: Fuerte Olimpo
Processing research for city: Ramnagar
Processing research for city: Boa Vista


In [19]:
hotel_names=[]

for x in range(len(results)):
    hotel=results[x]["results"][0]["name"]
    hotel_city=results[x]["results"][0]["plus_code"]["compound_code"]
    hotel_names.append(hotel)
    print(f'This is the hotel for the city {cities_list[x]}: {hotel}, located in {hotel_city}')
    print()

This is the hotel for the city Ambilobe: Hôtel & Restaurant DIANA Ambilobe, located in Q2WJ+9G Ambilobe, Madagascar

This is the hotel for the city Yanam: The Regency Hotel, located in P6J2+RR Yanam, Puducherry, India

This is the hotel for the city Cabo San Lucas: Hotel Tesoro Los Cabos, located in V3JQ+97 Cabo San Lucas, Baja California Sur, Mexico

This is the hotel for the city Fuerte Olimpo: Hotel Fuerte Olimpo, located in X45H+JQ Fuerte Olimpo, Paraguay

This is the hotel for the city Ramnagar: Samsara The Resort & Club, Corbett, located in 94CR+23 Kanchanpur, Uttarakhand, India

This is the hotel for the city Boa Vista: Aipana Plaza Hotel, located in R89G+HX Centro, Boa Vista - State of Roraima, Brazil



In [21]:
hotel_df["Hotel Name"]=hotel_names
hotel_df

<ipython-input-21-0b331b015f8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=hotel_names


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
15,Ambilobe,0,MG,1603598425,56,-13.20,49.05,25.39,2.29,Hôtel & Restaurant DIANA Ambilobe
28,Yanam,0,IN,1603598565,62,16.73,82.22,29.00,2.60,The Regency Hotel
246,Cabo San Lucas,0,MX,1603598354,77,22.89,-109.91,27.78,0.45,Hotel Tesoro Los Cabos
453,Fuerte Olimpo,0,PY,1603598739,72,-21.04,-57.87,25.61,1.72,Hotel Fuerte Olimpo
490,Ramnagar,0,IN,1603598757,37,29.40,79.12,25.92,0.22,"Samsara The Resort & Club, Corbett"
528,Boa Vista,0,BR,1603598740,66,2.82,-60.67,29.00,2.10,Aipana Plaza Hotel


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))